In [ ]:
import numpy as np
import pandas as pd
import sympy as sym
import math
from sympy.printing.theanocode import theano_function
import matplotlib.pyplot as plt
import utils
import _MEWtools as mt
import cycloidData

In [ ]:
interior = utils.import_interior('interior1')

TOLERANCE = 1

cycloids = cycloidData.loadAllCycloids()
highResCycloids = cycloidData.loadAllCycloids(pointsPerCurve=10000)

In [ ]:
import utils

overrides = {
    'obliquity': 0,
    'obliquityPhase': 0,
    'obliquityPhaseRate': 0,
    'spinRate': 0,
    'nonSynchronusRotationRate': 0,
    'librationAmplitude': 0,
    'librationPhase': 0,
    'librationFrequency': 0
}

sample_sat = utils.import_structure('Sample', overrides)

In [ ]:
r, θ, φ, t = sym.symbols('r θ φ t', real = True)
stress = theano_function([t, φ, θ], (sample_sat.PC1, sample_sat.PC2, sample_sat.PCΨ, sample_sat.PCΨ2), 
                    dims={t: 1, φ: 1, θ:1}, 
                    dtypes={t: 'float64', φ: 'float64', θ:'float64'})

fnp = sym.lambdify([t, φ, θ], sample_sat.PC1, 'numpy')

In [ ]:
# f(.2, 1, 1)

In [ ]:
SIZE = 18000000

europa_orbit_seconds = 85 * 3600

time = np.random.default_rng().uniform(0, europa_orbit_seconds, SIZE)
colat = np.random.default_rng().uniform(0, np.pi, SIZE)
lon = np.random.default_rng().uniform(0, 2*np.pi, SIZE)

In [ ]:
%timeit f(time, colat, lon)
%timeit fnp(time, colat, lon)

In [ ]:
results = stress(time, colat, lon)

In [ ]:
results[3].shape

In [ ]:
results = stress(time, colat, lon)
maxStress = np.maximum(results[0], results[1])
key = maxStress == results[0]
theta = np.select([key, np.invert(key)], [results[2], results[3]])

output = np.column_stack((maxStress, theta, colat, lon, time))
output

In [ ]:
df = pd.DataFrame(output, columns=['stress', 'heading', 'lat', 'lon', 'time'])

In [ ]:
def getMewStressField(times, colats, lons):
    results = stress(times, colat, lons)
    maxStress = np.maximum(results[0], results[1])
    key = maxStress == results[0]
    theta = np.select([key, np.invert(key)], [results[2], results[3]])

    return np.column_stack((maxStress, theta, colats, lons, times))

In [ ]:
curve = highResCycloids['delphi'].curve.copy()
curve['lon'] = np.radians(curve['lon'])
curve['lat'] = np.radians(curve['lat'])
coordinates = np.array(curve[['lon', 'lat']])
times = np.arange(0, 360)
combinations = np.concatenate(
    np.array([[(c[0], c[1], time) for time in times] for c in coordinates]))



In [ ]:
lons = combinations[:, 0:1].flatten()
lats = combinations[:, 1:2].flatten()
fullTimes = combinations[:, 2:3].flatten()

output = getMewStressField(fullTimes, lats, lons)
df = pd.DataFrame(output, columns=['stress', 'heading', 'lat', 'lon', 'time'])

In [ ]:
# filtered = df.loc[(df['lat'] == -1.190315) & (df['lon'] == 4.013036)]
filtered = df.loc[(df['time'] == 100)]
plt.plot(filtered['lon'], filtered['stress'])

In [ ]:
%timeit output = getMewStressField(fullTimes, lats, lons)